In [1]:
import sqlite3

In [1]:
from google.colab import files
uploaded = files.upload()

Saving database_new.sqlite to database_new.sqlite


In [2]:
conn = sqlite3.connect('/content/database_new.sqlite')

In [3]:
pd.read_sql(""" SELECT *
                FROM sqlite_master
                WHERE type='table';""",
           conn)
#8 tables

,type,name,tbl_name,rootpage,sql
0,table,sqlite_sequence,sqlite_sequence,4,"CREATE TABLE sqlite_sequence(name,seq)"
1,table,Player_Attributes,Player_Attributes,11,"CREATE TABLE ""Player_Attributes"" (\n\t`id`\tIN..."
2,table,Player,Player,14,CREATE TABLE `Player` (\n\t`id`\tINTEGER PRIMA...
3,table,Match,Match,18,CREATE TABLE `Match` (\n\t`id`\tINTEGER PRIMAR...
4,table,League,League,24,CREATE TABLE `League` (\n\t`id`\tINTEGER PRIMA...
5,table,Country,Country,26,CREATE TABLE `Country` (\n\t`id`\tINTEGER PRIM...
6,table,Team,Team,29,"CREATE TABLE ""Team"" (\n\t`id`\tINTEGER PRIMARY..."
7,table,Team_Attributes,Team_Attributes,2,CREATE TABLE `Team_Attributes` (\n\t`id`\tINTE...


In [5]:
pd.read_sql(""" SELECT *
                FROM Country;""",
           conn)
# 11 rows, name and id of country

,id,name
0,1,Belgium
1,1729,England
2,4769,France
3,7809,Germany
4,10257,Italy
5,13274,Netherlands
6,15722,Poland
7,17642,Portugal
8,19694,Scotland
9,21518,Spain


In [9]:
pd.read_sql(""" SELECT *
                FROM League;""",
           conn)
#11 leagues

,id,country_id,name
0,1,1,Belgium Jupiler League
1,1729,1729,England Premier League
2,4769,4769,France Ligue 1
3,7809,7809,Germany 1. Bundesliga
4,10257,10257,Italy Serie A
5,13274,13274,Netherlands Eredivisie
6,15722,15722,Poland Ekstraklasa
7,17642,17642,Portugal Liga ZON Sagres
8,19694,19694,Scotland Premier League
9,21518,21518,Spain LIGA BBVA


In [10]:
pd.read_sql(""" SELECT *
                FROM Match;""",
           conn)
#25000+rows, 115 cols with lots of NAN values

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_player_X1,home_player_X2,home_player_X3,home_player_X4,home_player_X5,home_player_X6,home_player_X7,home_player_X8,home_player_X9,home_player_X10,home_player_X11,away_player_X1,away_player_X2,away_player_X3,away_player_X4,away_player_X5,away_player_X6,away_player_X7,away_player_X8,away_player_X9,away_player_X10,away_player_X11,home_player_Y1,home_player_Y2,home_player_Y3,home_player_Y4,home_player_Y5,home_player_Y6,home_player_Y7,...,away_player_10,away_player_11,goal,shoton,shotoff,foulcommit,card,cross,corner,possession,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,LBH,LBD,LBA,PSH,PSD,PSA,WHH,WHD,WHA,SJH,SJD,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,None,None,None,None,None,None,None,None,1.73,3.40,5.00,1.75,3.35,4.20,1.85,3.2,3.5,1.80,3.3,3.75,NaN,NaN,NaN,1.70,3.30,4.33,1.90,3.3,4.00,1.65,3.40,4.50,1.78,3.25,4.00,1.73,3.40,4.20
1,2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,None,None,None,None,None,None,None,None,1.95,3.20,3.60,1.80,3.30,3.95,1.90,3.2,3.5,1.90,3.2,3.50,NaN,NaN,NaN,1.83,3.30,3.60,1.95,3.3,3.80,2.00,3.25,3.25,1.85,3.25,3.75,1.91,3.25,3.60
2,3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,8635,0,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,None,None,None,None,None,None,None,None,2.38,3.30,2.75,2.40,3.30,2.55,2.60,3.1,2.3,2.50,3.2,2.50,NaN,NaN,NaN,2.50,3.25,2.40,2.63,3.3,2.50,2.35,3.25,2.65,2.50,3.20,2.50,2.30,3.20,2.75
3,4,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9991,9998,5,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,None,None,None,None,None,None,None,None,1.44,3.75,7.50,1.40,4.00,6.80,1.40,3.9,6.0,1.44,3.6,6.50,NaN,NaN,NaN,1.44,3.75,6.00,1.44,4.0,7.50,1.45,3.75,6.50,1.50,3.75,5.50,1.44,3.75,6.50
4,5,1,1,2008/2009,1,2008-08-16 00:00:00,492477,7947,9985,1,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,None,None,None,None,None,None,None,None,5.00,3.50,1.65,5.00,3.50,1.60,4.00,3.3,1.7,4.00,3.4,1.72,NaN,NaN,NaN,4.20,3.40,1.70,4.50,3.5,1.73,4.50,3.40,1.65,4.50,3.50,1.65,4.75,3.30,1.67
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25974,25975,24558,24558,2015/2016,9,2015-09-22 00:00:00,1992091,10190,10191,1,0,1.0,2.0,4.0,6.0,8.0,4.0,6.0,3.0,5.0,7.0,5.0,1.0,2.0,4.0,6.0,8.0,4.0,6.0,3.0,5.0,7.0,5.0,1.0,3.0,3.0,3.0,3.0,6.0,6.0,...,232110.0,289732.0,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25975,25976,24558,24558,2015/2016,9,2015-09-23 00:00:00,1992092,9824,10199,1,2,1.0,3.0,5.0,7.0,2.0,4.0,6.0,8.0,5.0,4.0,6.0,1.0,2.0,4.0,6.0,8.0,3.0,5.0,7.0,3.0,5.0,7.0,1.0,3.0,3.0,3.0,7.0,7.0,7.0,...,34082.0,95257.0,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25976,25977,24558,24558,2015/2016,9,2015-09-23 00:00:00,1992093,9956,10179,2,0,1.0,2.0,4.0,6.0,8.0,4.0,6.0,3.0,5.0,7.0,5.0,1.0,2.0,4.0,6.0,8.0,4.0,6.0,3.0,5.0,7.0,5.0,1.0,3.0,3.0,3.0,3.0,6.0,6.0,...,451335.0,275122.0,None,None,None,None,None,None,None,None,NaN,NaN,NaN,NaN,NaN,Na

In [ ]:
pd.read_sql(""" SELECT *
                FROM Team;""",
           conn)
# 299 teams

,id,team_api_id,team_fifa_api_id,team_long_name,team_short_name
0,1,9987,673.0,KRC Genk,GEN
1,2,9993,675.0,Beerschot AC,BAC
2,3,10000,15005.0,SV Zulte-Waregem,ZUL
3,4,9994,2007.0,Sporting Lokeren,LOK
4,5,9984,1750.0,KSV Cercle Brugge,CEB
...,...,...,...,...,...
294,49479,10190,898.0,FC St. Gallen,GAL
295,49837,10191,1715.0,FC Thun,THU
296,50201,9777,324.0,Servette FC,SER
297,50204,7730,1862.0,FC Lausanne-Sports,LAU


In [38]:
CountryLeagueMatch = pd.read_sql("""SELECT League.name as league_name, Country.name as country_name, 
                                    Match.season, stage, Team.team_long_name as home_team_name, 
                                    away_info.away_team_name, Match.home_team_goal,	Match.away_team_goal 
                                    FROM League
                                    JOIN Country ON Country.id = League.country_id
                                    join Match on Match.country_id = League.country_id
                                    join Team on Team.team_api_id = Match.home_team_api_id
                                    join (SELECT Match.away_team_api_id , Team.team_long_name as away_team_name
                                    FROM Match
                                    join Team on Team.team_api_id = Match.away_team_api_id) as away_info
                                    on away_info.away_team_api_id = Match.away_team_api_id      
                                    order by league_name, season, home_team_name, away_team_name	             
                                              ;""", conn)
CountryLeagueMatch
#combine country, match, and league, and replace team id by team name

,league_name,country_name,season,stage,home_team_name,away_team_name,home_team_goal,away_team_goal
0,Belgium Jupiler League,Belgium,2008/2009,31,Beerschot AC,Club Brugge KV,2,0
1,Belgium Jupiler League,Belgium,2008/2009,31,Beerschot AC,Club Brugge KV,2,0
2,Belgium Jupiler League,Belgium,2008/2009,31,Beerschot AC,Club Brugge KV,2,0
3,Belgium Jupiler League,Belgium,2008/2009,31,Beerschot AC,Club Brugge KV,2,0
4,Belgium Jupiler League,Belgium,2008/2009,31,Beerschot AC,Club Brugge KV,2,0
...,...,...,...,...,...,...,...,...
2950450,Switzerland Super League,Switzerland,2015/2016,32,Lugano,Grasshopper Club Zürich,0,1
2950451,Switzerland Super League,Switzerland,2015/2016,32,Lugano,Grasshopper Club Zürich,0,1
2950452,Switzerland Super League,Switzerland,2015/2016,32,Lugano,Grasshopper Club Zürich,0,1
2950453,Switzerland Super League,Switzerland,2015/2016,32,Lugano,Grasshopper Club Zürich,0,1


In [91]:
CountryLeagueMatch = pd.read_sql("""SELECT League.name as league_name, Country.name as country_name, 
                                    Match.season, Team.team_long_name as home_team_name,
                                    round(avg(Match.home_team_goal),2) as avg_hometeam_goal,	
                                    round(avg(Match.away_team_goal),2) as avg_awayteam_goal,
                                    CASE WHEN round(avg(Match.home_team_goal),2) > round(avg(Match.away_team_goal),2)  then 'Win'
                                         WHEN round(avg(Match.home_team_goal),2) = round(avg(Match.away_team_goal),2)  then 'Draw'
                                         ELSE 'Lose'
                                         END AS 'Lose/Win'
                                    FROM League
                                    JOIN Country ON Country.id = League.country_id
                                    join Match on Match.country_id = League.country_id
                                    join Team on Team.team_api_id = Match.home_team_api_id
                                    where home_team_name in ('Beerschot AC')
                                    group by league_name, season
                                    order by league_name, Season            
                                                          ;""", conn)
CountryLeagueMatch
#find the average goal of Beerschot AC as home team in each season and compare the average home team goal with average away team goal
#using CASE to identify the status of the comparison

,league_name,country_name,season,home_team_name,avg_hometeam_goal,avg_awayteam_goal,Lose/Win
0,Belgium Jupiler League,Belgium,2008/2009,Beerschot AC,1.88,1.12,Win
1,Belgium Jupiler League,Belgium,2009/2010,Beerschot AC,1.43,1.57,Lose
2,Belgium Jupiler League,Belgium,2010/2011,Beerschot AC,0.93,0.67,Win
3,Belgium Jupiler League,Belgium,2011/2012,Beerschot AC,1.73,0.87,Win
4,Belgium Jupiler League,Belgium,2012/2013,Beerschot AC,1.33,2.27,Lose


In [ ]:
pd.read_sql(""" SELECT *
                FROM Team_Attributes;""",
           conn)
#1458 rows, may combine with other tables

,id,team_fifa_api_id,team_api_id,date,buildUpPlaySpeed,buildUpPlaySpeedClass,buildUpPlayDribbling,buildUpPlayDribblingClass,buildUpPlayPassing,buildUpPlayPassingClass,buildUpPlayPositioningClass,chanceCreationPassing,chanceCreationPassingClass,chanceCreationCrossing,chanceCreationCrossingClass,chanceCreationShooting,chanceCreationShootingClass,chanceCreationPositioningClass,defencePressure,defencePressureClass,defenceAggression,defenceAggressionClass,defenceTeamWidth,defenceTeamWidthClass,defenceDefenderLineClass
0,1,434,9930,2010-02-22 00:00:00,60,Balanced,NaN,Little,50,Mixed,Organised,60,Normal,65,Normal,55,Normal,Organised,50,Medium,55,Press,45,Normal,Cover
1,2,434,9930,2014-09-19 00:00:00,52,Balanced,48.0,Normal,56,Mixed,Organised,54,Normal,63,Normal,64,Normal,Organised,47,Medium,44,Press,54,Normal,Cover
2,3,434,9930,2015-09-10 00:00:00,47,Balanced,41.0,Normal,54,Mixed,Organised,54,Normal,63,Normal,64,Normal,Organised,47,Medium,44,Press,54,Normal,Cover
3,4,77,8485,2010-02-22 00:00:00,70,Fast,NaN,Little,70,Long,Organised,70,Risky,70,Lots,70,Lots,Organised,60,Medium,70,Double,70,Wide,Cover
4,5,77,8485,2011-02-22 00:00:00,47,Balanced,NaN,Little,52,Mixed,Organised,53,Normal,48,Normal,52,Normal,Organised,47,Medium,47,Press,52,Normal,Cover
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1453,1454,15005,10000,2011-02-22 00:00:00,52,Balanced,NaN,Little,52,Mixed,Organised,52,Normal,48,Normal,53,Normal,Organised,46,Medium,48,Press,53,Normal,Cover
1454,1455,15005,10000,2012-02-22 00:00:00,54,Balanced,NaN,Little,51,Mixed,Organised,47,Normal,52,Normal,50,Normal,Organised,44,Medium,55,Press,53,Normal,Cover
1455,1456,15005,10000,2013-09-20 00:00:00,54,Balanced,NaN,Little,51,Mixed,Organised,47,Normal,52,Normal,32,Little,Organised,44,Medium,58,Press,37,Normal,Cover
1456,1457,15005,10000,2014-09-19 00:00:00,54,Balanced,42.0,Normal,51,Mixed,Organised,47,Normal,52,Normal,32,Little,Organised,44,Medium,58,Press,37,Normal,Cover


In [30]:
teams_have_attributes = pd.read_sql(""" select count(number) from 
                             (SELECT count(*) as number
                             FROM Team_Attributes
                             group by team_api_id);""",
           conn)
teams_have_attributes
#only 288 teams have record of attributes while there totally 299 teams

,count(number)
0,288


In [80]:
pd.read_sql(""" SELECT Team.team_api_id, Team.team_long_name
                FROM Team
                join
                (SELECT Team.team_api_id
                FROM Team
                Except 
                select Team_Attributes.team_api_id 
                from Team_Attributes) as exception
                on Team.team_api_id = exception.team_api_id
                order by team_api_id;""",
           conn)
#extract the teams which didn't have attributes records

,team_api_id,team_long_name
0,4049,Tubize
1,4064,Feirense
2,6367,Uniao da Madeira
3,6601,FC Volendam
4,7896,Lugano
5,7947,FCV Dender EH
6,7992,Trofense
7,9765,Portimonense
8,10213,Amadora
9,177361,Termalica Bruk-Bet Nieciecza


In [83]:
teams_details = pd.read_sql("""SELECT avg(buildUpPlaySpeed) as avg_buildUpPlaySpeed, Team.team_api_id, 
                               Team.team_long_name from Team
                               left join Team_Attributes
                               on Team.team_api_id = Team_Attributes.team_api_id
                               group by Team.team_api_id
                               order by avg(buildUpPlaySpeed) desc
                               limit 5 ;""", conn)
teams_details
#top 5 team which got the fastest play speed

,avg_buildUpPlaySpeed,team_api_id,team_long_name
0,72.500000,9789,Borussia Dortmund
1,71.000000,8600,Udinese
2,70.000000,8596,Falkirk
3,67.000000,158085,FC Arouca
4,66.833333,8178,Bayer 04 Leverkusen


In [ ]:
pd.read_sql(""" SELECT *
                FROM Player;""",
           conn)
#11060 rows, 7cols, over 10k players? any overlap?

,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
0,1,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187
1,2,155782,Aaron Cresswell,189615,1989-12-15 00:00:00,170.18,146
2,3,162549,Aaron Doran,186170,1991-05-13 00:00:00,170.18,163
3,4,30572,Aaron Galindo,140161,1982-05-08 00:00:00,182.88,198
4,5,23780,Aaron Hughes,17725,1979-11-08 00:00:00,182.88,154
...,...,...,...,...,...,...,...
11055,11071,26357,Zoumana Camara,2488,1979-04-03 00:00:00,182.88,168
11056,11072,111182,Zsolt Laczko,164680,1986-12-18 00:00:00,182.88,176
11057,11073,36491,Zsolt Low,111191,1979-04-29 00:00:00,180.34,154
11058,11074,35506,Zurab Khizanishvili,47058,1981-10-06 00:00:00,185.42,172


In [ ]:
pd.read_sql(""" SELECT *
                FROM Player_Attributes;""",
           conn)
#180k rows, 42 cols

,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,free_kick_accuracy,long_passing,ball_control,acceleration,sprint_speed,agility,reactions,balance,shot_power,jumping,stamina,strength,long_shots,aggression,interceptions,positioning,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,1,218353,505942,2016-02-18 00:00:00,67.0,71.0,right,medium,medium,49.0,44.0,71.0,61.0,44.0,51.0,45.0,39.0,64.0,49.0,60.0,64.0,59.0,47.0,65.0,55.0,58.0,54.0,76.0,35.0,71.0,70.0,45.0,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,2,218353,505942,2015-11-19 00:00:00,67.0,71.0,right,medium,medium,49.0,44.0,71.0,61.0,44.0,51.0,45.0,39.0,64.0,49.0,60.0,64.0,59.0,47.0,65.0,55.0,58.0,54.0,76.0,35.0,71.0,70.0,45.0,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,3,218353,505942,2015-09-21 00:00:00,62.0,66.0,right,medium,medium,49.0,44.0,71.0,61.0,44.0,51.0,45.0,39.0,64.0,49.0,60.0,64.0,59.0,47.0,65.0,55.0,58.0,54.0,76.0,35.0,63.0,41.0,45.0,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,4,218353,505942,2015-03-20 00:00:00,61.0,65.0,right,medium,medium,48.0,43.0,70.0,60.0,43.0,50.0,44.0,38.0,63.0,48.0,60.0,64.0,59.0,46.0,65.0,54.0,58.0,54.0,76.0,34.0,62.0,40.0,44.0,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,5,218353,505942,2007-02-22 00:00:00,61.0,65.0,right,medium,medium,48.0,43.0,70.0,60.0,43.0,50.0,44.0,38.0,63.0,48.0,60.0,64.0,59.0,46.0,65.0,54.0,58.0,54.0,76.0,34.0,62.0,40.0,44.0,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183973,183974,102359,39902,2009-08-30 00:00:00,83.0,85.0,right,medium,low,84.0,77.0,59.0,89.0,77.0,84.0,86.0,78.0,84.0,85.0,66.0,72.0,77.0,86.0,73.0,76.0,58.0,72.0,67.0,81.0,56.0,78.0,86.0,88.0,83.0,22.0,31.0,30.0,9.0,20.0,84.0,20.0,20.0
183974,183975,102359,39902,2009-02-22 00:00:00,78.0,80.0,right,medium,low,74.0,76.0,53.0,84.0,77.0,85.0,86.0,74.0,73.0,86.0,66.0,67.0,77.0,74.0,73.0,75.0,58.0,66.0,65.0,73.0,61.0,64.0,72.0,88.0,70.0,32.0,31.0,30.0,9.0,20.0,73.0,20.0,20.0
183975,183976,102359,39902,2008-08-30 00:00:00,77.0,80.0,right,medium,low,74.0,71.0,53.0,84.0,77.0,85.0,86.0,74.0,73.0,86.0,66.0,67.0,77.0,74.0,73.0,75.0,58.0,66.0,65.0,73.0,67.0,64.0,72.0,88.0,70.0,32.0,31.0,30.0,9.0,20.0,73.0,20.0,20.0
183976,183977,102359,39902,2007-08-30 00:00:00,78.0,81.0,right,medium,low,74.0,64.0,57.0,86.0,77.0,87.0,86.0,73.0,73.0,91.0,61.0,60.0,77.0,69.0,73.0,72.0,58.0,67.0,59.0,78.0,63.0,63.0,68.0,88.0,53.0,28.0,32.0,30.0,9.0,20.0,73.0,20.0,20.0


In [26]:
pd.read_sql(""" SELECT Player.player_api_id, player_name, date, height, 
                max(overall_rating) as max_rating
                FROM Player
                join
                (select Player_Attributes.player_api_id  as player_id, date, 
                 overall_rating from Player_Attributes
                 where date >= '2010-01-01 00:00:00') as date_selection
                on date_selection.player_id = Player.player_api_id
                group by player_name
                having height >= 180
                order by max_rating desc, height desc
                limit 10
               ;""",
           conn)

#find top 10 max score graded after 2010 (players are higher than 180）and the exact time graded

,player_api_id,player_name,date,height,max_rating
0,30893,Cristiano Ronaldo,2015-10-16 00:00:00,185.42,93
1,35724,Zlatan Ibrahimovic,2014-10-17 00:00:00,195.58,90
2,27299,Manuel Neuer,2016-04-21 00:00:00,193.04,90
3,30657,Iker Casillas,2010-02-22 00:00:00,185.42,90
4,40636,Luis Suarez,2015-10-16 00:00:00,182.88,90
5,30834,Arjen Robben,2015-09-25 00:00:00,180.34,90
6,30865,Nemanja Vidic,2012-08-31 00:00:00,190.50,89
7,30843,Robin van Persie,2014-05-23 00:00:00,187.96,89
8,30989,Julio Cesar,2010-02-22 00:00:00,185.42,89
9,37482,Gerard Pique,2012-02-22 00:00:00,193.04,88
